<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Hero-Stats-Transformer" data-toc-modified-id="Hero-Stats-Transformer-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Hero Stats Transformer</a></span></li><li><span><a href="#Partidas-Dota-2---Transfomador" data-toc-modified-id="Partidas-Dota-2---Transfomador-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Partidas Dota 2 - Transfomador</a></span></li><li><span><a href="#Join-entre-dataframes" data-toc-modified-id="Join-entre-dataframes-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Join entre dataframes</a></span><ul class="toc-item"><li><span><a href="#Filtrando-game_mode-e-lobby_type" data-toc-modified-id="Filtrando-game_mode-e-lobby_type-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Filtrando game_mode e lobby_type</a></span></li><li><span><a href="#Unificando-datastes" data-toc-modified-id="Unificando-datastes-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Unificando datastes</a></span></li></ul></li><li><span><a href="#Salvando-dataset" data-toc-modified-id="Salvando-dataset-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Salvando dataset</a></span></li></ul></div>

---
# Transformador 1: Anexar os status de herói aos dados disponíveis

**Objetivo do experimento:** Anexar os status de cada herói ao dataset de treinamento. Ao final o dataset deve possuir mais de 500 features. Abaixo a *Ordem de procedimentos:*

**Arquivo Hero_stats**
1. Carregar o arquivo hero_stats
2. Aplicar a transformação nos dados de hero_stats

**Arquivos de partidas**
1. Procurar por todos os dados disponíveis na pasta 'raw_data'
2. Concatenar todos em um único dataframe
3. Remover valores nulos
4. Remover colunas duplicadas
5. Adicionar uma coluna para um herói de cada time

**União de dados**
1. Filtrar apenas por partidas rankeadas e game_mode = 'game_mode_all_draft'
2. Fazer um join entre os datasets

**Loading libraries and ``hero_stats`` JSON**

In [1]:
data_path = '../data/raw_data'

import pandas as pd
from datetime import datetime
import os

import sys
sys.path.append('../utils')
import transformer_utils

## Hero Stats Transformer

In [2]:
hero_stats_raw = pd.read_json(data_path+'/hero_stats.json')
hero_stats_df = transformer_utils.hero_stats_tranformer(hero_stats_raw)
hero_stats_df.head()

,localized_name,is_Melle,base_mana_regen,base_armor,base_attack_min,base_attack_max,base_str,base_agi,base_int,str_gain,...,primary_attr_str,Nuker,Disabler,Initiator,Durable,Support,Jungler,Carry,Pusher,Escape
0,Anti-Mage,1,0.0,0.0,29,33,23,24,12,1.3,...,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,Axe,1,0.0,-1.0,27,31,25,20,18,3.4,...,1,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
2,Bane,0,0.0,1.0,35,41,22,22,22,2.6,...,0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,Bloodseeker,1,0.0,2.0,35,41,24,22,17,2.7,...,0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
4,Crystal Maiden,0,1.0,-1.0,28,34,18,16,16,2.2,...,0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [3]:
# Removing hero name
hero_stats_df.drop(columns=['localized_name'], inplace=True)

In [4]:
hero_stats_df.columns

Index(['is_Melle', 'base_mana_regen', 'base_armor', 'base_attack_min',
       'base_attack_max', 'base_str', 'base_agi', 'base_int', 'str_gain',
       'agi_gain', 'int_gain', 'attack_range', 'projectile_speed',
       'attack_rate', 'move_speed', 'legs', 'hero_id', 'turbo_picks',
       'turbo_wins', 'pro_win', 'pro_pick', 'pro_ban', '1_pick', '1_win',
       '2_pick', '2_win', '3_pick', '3_win', '4_pick', '4_win', '5_pick',
       '5_win', '6_pick', '6_win', '7_pick', '7_win', '8_pick', '8_win',
       'null_pick', 'primary_attr_agi', 'primary_attr_int', 'primary_attr_str',
       'Nuker', 'Disabler', 'Initiator', 'Durable', 'Support', 'Jungler',
       'Carry', 'Pusher', 'Escape'],
      dtype='object')

## Partidas Dota 2 - Transfomador 

**Searching for each .csv file in the 'raw_data' folder**

In [5]:
lst_df = []
for root, dirs, files in os.walk(data_path):
    for filename in files:
        xlsx_file, file_extension = os.path.splitext(filename)
        print('.csv file found:\n')
        if(file_extension == '.csv'):
            print(filename)
            file_path = root +'\\' + filename
            lst_df.append(pd.read_csv(file_path))  

.csv file found:

21-04-24 16h14m31s.csv
.csv file found:

.csv file found:

.csv file found:



**Drop NA**

In [6]:
final_df = pd.concat(lst_df)

print('Dataframe shape:', final_df.shape)
print('Total nan: \n\n', final_df.isna().sum())

final_df.dropna(inplace=True)
print('\nDataframe shape:', final_df.shape)

Dataframe shape: (46000, 9)
Total nan: 

 Unnamed: 0      0
match_id        0
radiant_win     0
avg_mmr         0
duration        0
lobby_type      0
game_mode       0
radiant_team    0
dire_team       0
dtype: int64

Dataframe shape: (46000, 9)


**Remove duplicated rows**

In [7]:
final_df.drop_duplicates(subset=['match_id'])
print('\nDataframe shape:', final_df.shape)


Dataframe shape: (46000, 9)


In [ ]:
final_df

**Adding column for each hero on radiant and dire team**

In [ ]:
radiant_columns = ['Radiant 1','Radiant 2', 'Radiant 3', 'Radiant 4', 'Radiant 5']
dire_columns = ['Dire 1','Dire 2', 'Dire 3', 'Dire 4', 'Dire 5']

final_df[radiant_columns] = final_df.radiant_team.str.split(",",expand=True,)
final_df[dire_columns] = final_df.dire_team.str.split(",",expand=True,)
print('\nDataframe shape:', final_df.shape)

In [ ]:
final_df

**Removing Unnamed and dire_team and radiant_team column**

In [ ]:
final_df.drop(columns=['Unnamed: 0', 'dire_team', 'radiant_team', 'match_id'], inplace=True)
print('\nDataframe shape:', final_df.shape)

**Converting string to numerical**

In [ ]:
final_df['radiant_win'] = final_df['radiant_win'].astype(int)
final_df = final_df.apply(pd.to_numeric)
final_df.head()

## Join entre dataframes

### Filtrando game_mode e lobby_type

In [ ]:
game_mode_df = pd.read_json(data_path+'/game_mode.json').T
lobby_type_df = pd.read_json(data_path+'/lobby_type.json').T

dict_game_mode = dict(zip(game_mode_df['id'], game_mode_df['name']))
dict_lobby_type = dict(zip(lobby_type_df['id'], lobby_type_df['name']))

df = final_df

df['game_mode'].replace(dict_game_mode, inplace=True)
df['lobby_type'].replace(dict_lobby_type, inplace=True)


print('Formato do dataframe:', df.shape)
df = df[df['game_mode'] == 'game_mode_all_draft']
df = df[df['lobby_type'] == 'lobby_type_ranked']


df.drop(columns=['game_mode','lobby_type'], inplace=True)
print('Formato do dataframe após remoção:', df.shape)
final_df = df

In [ ]:
print(final_df.shape)
print(hero_stats_df.shape)

### Unificando datastes

In [ ]:
team_hero_names = ['Radiant 1','Radiant 2', 'Radiant 3', 'Radiant 4', 'Radiant 5',
                  'Dire 1','Dire 2', 'Dire 3', 'Dire 4', 'Dire 5']

team_prexis_names = ['Rdt_1_','Rdt_2_', 'Rdt_3_', 'Rdt_4_', 'Rdt_5_',
                  'Dire_1_','Dire_2_', 'Dire_3_', 'Dire_4_', 'Dire_5_']

result = pd.DataFrame()
for i,prefix in enumerate(team_prexis_names):
    aux_hero_stats = hero_stats_df
    aux_hero_stats = aux_hero_stats.add_prefix(prefix)

    pre_result = pd.merge(final_df, aux_hero_stats, left_on=team_hero_names[i], right_on= prefix+'hero_id')
    result = pd.concat([result, pre_result], axis=1)

In [ ]:
result

In [ ]:
# Removing id columns
print(result.shape)

for i,prefix in enumerate(team_prexis_names):
    result.drop(columns=[prefix+'hero_id', team_hero_names[i]], inplace=True)
    
print(result.shape)

In [ ]:
result.head()

In [ ]:
result

In [ ]:
result.describe().loc['std',:].plot(ylim=0,100)

## Salvando dataset
**Saving data frame on 'working data' folder**

In [ ]:
working_data_path = '../data/working_data/1_TRA_'
start_file = datetime.now().strftime("%Y-%m-%d")
output_file = working_data_path + start_file + '_working_data.csv'

result.to_csv(output_file, index=False)